In [ ]:
# !pip install pdfminer.six
# !pip install spacy 
# !python3 -m spacy download en_core_web_sm

In [2]:
import spacy,re,glob
import pdfminer.high_level as pdfhl
import pdfminer.layout as pdflo
import pandas as pd

# 设置extract_text()的参数，未设置部分单词会存在没有空格的情况
laparams = pdflo.LAParams()
setattr(laparams, 'all_texts', True)
# 导入data目录下的所有pdf
pdfs = glob.glob("data/*.pdf")
text = ""
for pdf in pdfs:
    temp = pdfhl.extract_text(pdf, laparams=laparams).lower()
    # 将无意义的字符（数字、特殊符号、缩写等）替换成空格
    temp = re.sub(r"burningvocabulary\.com|’ll|n’t|’s|’re|[^a-zA-Z]+",' ',temp)
    text += temp


# 通过Spacy 还原单词原型，并统计单词的出现频次
nlp = spacy.load("en_core_web_sm")
# 默认的nlp处理的文本大小1000000,报错的话 把max_length设置大一点
nlp.max_length = 2000000
doc = nlp(text)
dicts = {}
for token in doc:
    # 只统计字母数大于2的单词
    if len(token.lemma_) > 2:
        dicts[token.lemma_] = dicts.get(token.lemma_,0)+1
    

In [3]:

# 转置，导入pandas 进行分析
df=pd.DataFrame([dicts]).T
df=df.reset_index().rename(columns={'index':'word',0:'count'})
df.sort_values(by="count",ascending=False)

,word,count
8,the,8279
15,and,3039
54,that,2018
119,have,1448
4,for,1403
...,...,...
3522,bloodstream,1
6594,insightful,1
6595,provocative,1
3521,newborn,1


In [57]:

# 导入高中单词
with open('data/highSchoolWords.csv', 'r', encoding='utf-8') as f:
    content = f.read().lower()
result = re.findall(r"(?:^|(?:\n))(\w+)",content)
highSchoolWords = pd.DataFrame(result,columns=["word"])


# 过滤 属于高中的单词 （ ～ 代表非）
data = df[~df.word.isin(highSchoolWords.word)]

# 查找词频大于50的单词并按降序排序
d = data.query("count >=3").sort_values(by="count", ascending=False)
# 保存单词
d["word"].to_csv("words.csv",index=0)

print(d.shape)
d



(1999, 2)


,word,count
560,american,116
533,economic,102
1032,researcher,83
2594,federal,79
1199,don,76
...,...,...
5131,attorney,3
5132,homebuying,3
5134,hampson,3
5139,purchasing,3
